In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
import joblib
from hsml.model_schema import ModelSchema
from hsml.schema import Schema
from sklearn.metrics import mean_absolute_error

import src.config as config
from src.data_utils import transform_ts_data_info_features_and_target
from src.inference import (
    fetch_days_data,
    get_hopsworks_project,
    load_metrics_from_registry,
    load_model_from_registry,
)
from src.pipeline_utils import get_pipeline

In [3]:
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
import hopsworks
import numpy as np
import pandas as pd
from hsfs.feature_store import FeatureStore

import src.config as config
from src.data_utils import transform_ts_data_info_features

In [4]:
def get_feature_store() -> FeatureStore:
    project = get_hopsworks_project()
    return project.get_feature_store()

In [5]:
def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc)).tz_convert("America/New_York")
    fetch_data_from = current_date - timedelta(days=(days))
    fetch_data_to = current_date - timedelta(days=0)
    print(fetch_data_from, fetch_data_to)
    print("ghcgchcgfc")
    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_NAME, version=1)

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    print(sum(cond))
    return df[cond]
    # return df

In [6]:
print(f"Fetching data from group store ...")
ts_data = fetch_days_data(180)

Fetching data from group store ...
2024-11-12 09:26:23.123385-05:00 2025-05-11 10:26:23.123385-04:00
ghcgchcgfc
2025-05-11 10:26:23,206 INFO: Initializing external client
2025-05-11 10:26:23,206 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 10:26:24,050 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
55859


In [7]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-05-06 14:00:00+00:00,HB305,0
1,2025-04-26 09:00:00+00:00,HB608,0
2,2025-04-23 08:00:00+00:00,HB608,0
3,2025-04-24 19:00:00+00:00,JC023,0
4,2025-05-07 08:00:00+00:00,JC035,0
...,...,...,...
55854,2025-04-18 09:00:00+00:00,JC115,0
55855,2025-05-08 00:00:00+00:00,JC018,1
55856,2025-04-22 09:00:00+00:00,HB611,0
55857,2025-04-28 19:00:00+00:00,HB304,0
